In [1]:
print("""
@Description: 一些代码运行草稿
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-07-28 11:26:10
""")


@Description: 一些代码运行草稿
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-07-28 11:26:10



### 4.1.2Embedding层的实现

In [2]:
import numpy as np
W = np.arange(21).reshape(7, 3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [3]:
W[2]

array([6, 7, 8])

In [4]:
W[5]

array([15, 16, 17])

从权重 $\bm{W}$ 中一次性提取多行的处理也很简单。

In [5]:
idx = np.array([1, 0, 3, 0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [6]:
W = np.arange(12).reshape(3, 4)
W

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [7]:
np.add.at(W, [0, 1, 1], 10)
W

array([[10, 11, 12, 13],
       [24, 25, 26, 27],
       [ 8,  9, 10, 11]])

#### Embedding 层 forward 演示

In [23]:
import sys
sys.path.append('..')
from common.layers import Embedding
W = np.random.randn(7, 3)
print(W)
idx = [0, 3, 1]
embed = Embedding(W)
target_W = embed.forward(idx)
print(target_W)
# h 是不是也是选出来的，类似于 embed.forward(idx)
h = np.arange(9).reshape(3, 3)
out = np.sum(target_W * h, axis=1)
print(out)

[[-0.30413609  0.48506527  0.66953897]
 [ 0.34318588  0.32782055 -1.29065113]
 [-1.49880446 -1.2044469  -0.56687515]
 [ 1.20338375  0.44910519  1.49058455]
 [ 0.48869693 -2.03302988 -1.57485321]
 [ 1.48964961 -2.97889991  0.10268561]
 [ 0.75619985 -0.7539774  -1.22005108]]
[[-0.30413609  0.48506527  0.66953897]
 [ 1.20338375  0.44910519  1.49058455]
 [ 0.34318588  0.32782055 -1.29065113]]
[ 1.82414321 12.85949475 -5.97134989]


### 基于概率分布的采样

In [24]:
np.random.choice(10)

3

In [26]:
np.random.choice(10)

9

In [27]:
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

'hello'

In [28]:
np.random.choice(words, size=5)

array(['you', '.', 'goodbye', 'say', 'say'], dtype='<U7')

In [29]:
np.random.choice(words, size=5, replace=False)

array(['say', '.', 'I', 'you', 'goodbye'], dtype='<U7')

In [30]:
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p)

'you'

In [32]:
p = [.7, .27, .01]
new_p = np.power(p, .75)
new_p /= np.sum(new_p)
new_p

array([0.6532697 , 0.3197362 , 0.02699411])

In [33]:
a = [0, 1, 0, 1, 1, 1, 2]
from collections import Counter

In [37]:
np.array(Counter(a).values())

array(dict_values([2, 4, 1]), dtype=object)

In [45]:
np.array(list(Counter(a).values()))

array([2, 4, 1])

In [50]:
counts = Counter()
for word_id in a:
    counts[word_id] += 1
# 不是与下面这一行等价？
# counts = Counter(corpus)
vocab_size = len(counts)
vocab_size = vocab_size

word_p = np.zeros(vocab_size)
for i in range(vocab_size):
    word_p[i] = counts[i]

In [51]:
counts

Counter({0: 2, 1: 4, 2: 1})

In [53]:
word_p/np.sum(word_p)

array([0.28571429, 0.57142857, 0.14285714])

### 4.2.6负采样的采样方法

In [77]:
from negative_sampling_layer import UnigramSampler

corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2
sampler = UnigramSampler(corpus, sample_size, power)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[3 4]
 [0 2]
 [1 4]]


In [91]:
negative = np.zeros((3, sample_size), dtype=np.int64)
for i in range(2):
    negative[i, :] = np.random.choice(5, size=sample_size, replace=False, p=sampler.word_p)
negative

array([[0, 4],
       [3, 1],
       [0, 0]], dtype=int64)